# Data Science Nanodegree - Blog Post
Created By: Renan Vieira Dias

### Imports

In [63]:
import pandas as pd
import math

### Reading Data Files

In [65]:
df = pd.read_csv('survey_results_public.csv')
df_schema = pd.read_csv('survey_results_schema.csv')
df_schema.head(3)

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?


In [68]:
df.shape

(88883, 85)

In [66]:
df.head(2)

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult


### Checking Every Work Title
#### Columns: DevType
#### Question: Which of the following describe you? Please select all that apply.
- Academic researcher
- Data or business analyst
- Data scientist or machine learning specialist
- Database administrator
- Designer
- Developer, backend
- Developer, desktop or enterprise applications
- Developer, embedded applications or devices
- Developer, frontend
- Developer, fullstack
- Developer, game or graphics
- Developer, mobile
- Developer, QA or test
- DevOps specialist
- Educator
- Engineer, data
- Engineer, site reliability
- Engineering manager
- Marketing or sales professional
- Product manager
- Scientist
- Senior Executive (CSuite,VP, etc.)
- Student
- System administrator
- Other

In [70]:
devtype_options = [
    'Academic researcher'
    ,'Data or business analyst'
    ,'Data scientist or machine learning specialist'
    ,'Database administrator'
    ,'Designer'
    ,'Developer, backend'
    ,'Developer, desktop or enterprise applications'
    ,'Developer, embedded applications or devices'
    ,'Developer, frontend'
    ,'Developer, fullstack'
    ,'Developer, game or graphics'
    ,'Developer, mobile'
    ,'Developer, QA or test'
    ,'DevOps specialist'
    ,'Educator'
    ,'Engineer, data'
    ,'Engineer, site reliability'
    ,'Engineering manager'
    ,'Marketing or sales professional'
    ,'Product manager'
    ,'Scientist'
    ,'Senior Executive (CSuite,VP, etc.)'
    ,'Student'
    ,'System administrator'
    ,'Other'
]

In [71]:
def transformingColumnByOptions(df, column, options, separator='_'):
    for each in options:
        new_column = column + separator + each
        df[new_column] = df[column].apply(lambda x: 1 if x == each else 0)
    return df

In [73]:
df_devtypes = transformingColumnByOptions( df, 'DevType', devtype_options)
df_devtypes.head(2)

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,"DevType_Engineer, data","DevType_Engineer, site reliability",DevType_Engineering manager,DevType_Marketing or sales professional,DevType_Product manager,DevType_Scientist,"DevType_Senior Executive (CSuite,VP, etc.)",DevType_Student,DevType_System administrator,DevType_Other
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,0,0,0,0,0,0,0,0,0,0


In [74]:
df_devtypes.describe()

,Respondent,CompTotal,ConvertedComp,WorkWeekHrs,CodeRevHrs,Age,DevType_Academic researcher,DevType_Data or business analyst,DevType_Data scientist or machine learning specialist,DevType_Database administrator,...,"DevType_Engineer, data","DevType_Engineer, site reliability",DevType_Engineering manager,DevType_Marketing or sales professional,DevType_Product manager,DevType_Scientist,"DevType_Senior Executive (CSuite,VP, etc.)",DevType_Student,DevType_System administrator,DevType_Other
count,88883.000000,5.594500e+04,5.582300e+04,64503.000000,49790.000000,79210.000000,88883.000000,88883.000000,88883.000000,88883.000000,...,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.0,88883.000000,88883.000000,88883.0
mean,44442.000000,5.519014e+11,1.271107e+05,42.127197,5.084308,30.336699,0.003296,0.004714,0.005918,0.001024,...,0.003229,0.001159,0.004624,0.000765,0.001913,0.001553,0.0,0.017045,0.003195,0.0
std,25658.456325,7.331926e+13,2.841523e+05,37.287610,5.513931,9.178390,0.057320,0.068497,0.076700,0.031981,...,0.056732,0.034022,0.067843,0.027649,0.043692,0.039373,0.0,0.129439,0.056436,0.0
min,1.000000,0.000000e+00,0.000000e+00,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
25%,22221.500000,2.000000e+04,2.577750e+04,40.000000,2.000000,24.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
50%,44442.000000,6.200000e+04,5.728700e+04,40.000000,4.000000,29.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
75%,66662.500000,1.200000e+05,1.000000e+05,44.750000,6.000000,35.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
max,88883.000000,1.000000e+16,2.000000e+06,4850.000000,99.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0


### Interest In DevTypes
- 'Data or business analyst'
- 'Data scientist or machine learning specialist'
- 'Database administrator'
- 'Engineer, data'

In [75]:
devtype_interest = [
    'Data or business analyst'
    ,'Data scientist or machine learning specialist'
    ,'Database administrator'
    ,'Engineer, data'
]
devtype_interest

['Data or business analyst',
 'Data scientist or machine learning specialist',
 'Database administrator',
 'Engineer, data']

In [85]:
def isOneOfTheList(x, list_options):
    for each in list_options:
        if each in x:
            return True
    return False

In [86]:
df_devtypes['DevType'].isnull().sum()

7548

In [95]:
# Clearing those with no answer of DevType
df_datajobs = df_devtypes.dropna(subset = ['DevType']).copy()
df_datajobs['datajobs'] = df_datajobs['DevType'].apply( isOneOfTheList, list_options = df_devtypeInterest)
df_datajobs = df_datajobs[df_datajobs['datajobs']]
df_datajobs = df_datajobs.drop(['datajobs'], axis=1)
df_datajobs.head(2)

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,"DevType_Engineer, data","DevType_Engineer, site reliability",DevType_Engineering manager,DevType_Marketing or sales professional,DevType_Product manager,DevType_Scientist,"DevType_Senior Executive (CSuite,VP, etc.)",DevType_Student,DevType_System administrator,DevType_Other
5,6,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Canada,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,...,0,0,0,0,0,0,0,0,0,0
8,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,0,0,0,0,0,0,0,0,0,0


In [96]:
df_datajobs.shape

(19752, 110)